# Intro. ML Final Project

By Dave Lu, Alina Chen, Jin Rong Song

In [14]:
import pandas as pd
# import sklearn
# import torch

Loading the data into Numpy arrays.

In [15]:
data = pd.read_csv("https://raw.githubusercontent.com/daveylu/10315-project/main/data.csv")

x = data[["Variance", "Skewness", "Curtosis", "Entropy"]]
y = data["Class"]

x = x.to_numpy()
y = y.to_numpy()